In [1]:
import numpy as np
import json
import os

In [2]:
# read multiplication table file
infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '\z2_s4_001.json', 'r')]
mixer_index = infile[0]
M = infile[1] #multipliers
T = infile[2] #multiplication table

assert T.shape[0] == T.shape[1] == T.shape[2]
len_T = T.shape[0]
print("Number of conjugacy classes: {}" .format(len_T))


assert M.shape[1] == len_T
len_M = M.shape[0]
print("Number of multipliers: {}" .format(len_M))

#print("Multiplier Indices: {}" .format(M))

Number of conjugacy classes: 33
Number of multipliers: 9


In [3]:
# Convert multplication table from 3D array of integers to 3D list of bin
def array_to_list(X):
    return X.tolist()

M1 = array_to_list(M)
T1 = array_to_list(T)

In [15]:
nbytes = 2
PAD = '0'

def pad_string(m, len_T, PAD):
    pad = PAD*(len_T - len(m) % len_T)
    return m + pad

def string_to_byte(m, nbytes):
    #convert string to binary sequence
    #print(bin_version)
    return ''.join(format(ord(i), '08b') for i in m)


def byte_to_binary(m, len_T, nbytes):
    #seperate byte sequence into list of 'chunks' of length 'bits_per_coefficient'
    bits_per_coefficient = nbytes * 2
    res = []
    for start in range(0, len(m), bits_per_coefficient):
        res.append(m[start:start + bits_per_coefficient])
    res = [bin(int(e,2)) for e in res]
    #seperate chunks into lists of length 'len_T'
    return [res[i:i + len_T] for i in range(0, len(res), len_T)]

def binary_to_int(m):
    res = []
    for i in range(len(m)):
        res[i] = [int(e,2) for e in m[i]]
    return res

def binary_to_byte(m, nbytes):
    res = ''
    for i in range(len(m)):
        for j in range(len(m[i])):
            byte = m[i][j][2:].zfill(nbytes*2)
            res += byte
    return res

def byte_to_string(m):
    return ''.join(chr(int(m[i*8:i*8+8],2)) for i in range(len(m)//8))

In [8]:
# Demonstration
m = 'abcdefg'
print("String: %s; Length: %s" % (m, len(m)))
m1 = pad_string(m, len_T, PAD)
print("Padded String: %s; Length: %s" % (m1, len(m1)))
m2 = string_to_byte(m1, nbytes)
print("Byte Sequence: %s; Length: %s" % (m2, len(m2)))
m3 = byte_to_binary(m2, len_T, nbytes)
print("Binary Sequence: %s" % (m3))
m4 = binary_to_byte(m3, nbytes)
print("Byte Sequence: %s" % (m4))
m5 = byte_to_string(m4)
print("Reconstructed String: %s" % (m5))

String: abcdefg; Length: 7
Padded String: abcdefg00000000000000000000000000; Length: 33
Byte Sequence: 011000010110001001100011011001000110010101100110011001110011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000; Length: 264
Binary Sequence: [['0b110', '0b1', '0b110', '0b10', '0b110', '0b11', '0b110', '0b100', '0b110', '0b101', '0b110', '0b110', '0b110', '0b111', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11'], ['0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0', '0b11', '0b0']]
Byte Sequence: 0110000101100010011000110110010001100101011001100110011100110000001100

TypeError: int() can't convert non-string with explicit base

In [96]:
def brmult_list(T, L, sel = "none"):
    if sel == "none":
        pass
    else:
        L = L[sel,:]
        
    for l in L:
        if all(l == L[0]): # initialize product
            chi = l
        else:
            temp = np.zeros(len(l))
            for i in range(len(l)):
                for j in range(len(l)):
                    temp+= chi[i]*l[j]*T[i][j]
            chi = temp
            #print("-------------------------------------------------------------")
            #print(chi) 
    return chi

In [97]:
msg = "abcdefghijklmnopqrs"
msg_utf = encode_msg(msg, len_T)
print(msg_utf)

[array([ 110,    1,  110,   10,  110,   11,  110,  100,  110,  101,  110,
        110,  110,  111,  110, 1000,  110, 1001,  110, 1010,  110, 1011,
        110, 1100,  110, 1101,  110, 1110,  110, 1111,  111,    0,  111]), array([  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   1, 111,  10, 111,  11])]


In [103]:
bin_msg = ''.join(format(ord(i), '08b') for i in msg)
bin_msg
#''.join(chr(int(msg[i*8:i*8+8]),2) for i in range(len(msg)//8))

'01100001011000100110001101100100011001010110011001100111011010000110100101101010011010110110110001101101011011100110111101110000011100010111001001110011'

In [ ]:
def to_bytestr(m, nbytes =2):
    return ''.join(chr(int(m[i*8:i*8+8]),2) for i in range(len(m)//8))

In [98]:
multiplier = brmult_list(T,M)
encryption = [brmult_list(T,[e,multiplier]) for e in msg_utf]
#encryption = brmult_list(T,[msg_utf,multiplier])
print(encryption)

[array([ 2.222e+03, -1.333e+03,  3.654e+03, -1.010e+03, -2.442e+03,
       -2.343e+03, -2.210e+02, -2.432e+03, -2.210e+02, -1.533e+03,
       -1.452e+03, -1.541e+03, -2.532e+03,  2.553e+03, -2.443e+03,
        1.000e+03,  1.000e+00, -8.900e+02,  1.000e+00, -8.990e+02,
        1.000e+00,  1.232e+03,  3.310e+02,  1.321e+03,  1.000e+00,
        1.322e+03,  1.000e+00, -1.332e+03, -3.320e+02, -1.222e+03,
       -2.220e+02, -1.110e+02,  1.110e+02]), array([  15.,  -17.,   30.,   -4.,  -19.,  -19.,  -13.,  -18.,  -13.,
        -18.,  -18.,  -18.,  -18.,   28.,  -18.,    2.,    9.,    9.,
          9.,    9.,    9.,   14.,   14.,   14.,    9.,   14.,    9.,
        -24.,  -23., -122.,  -21., -122.,   11.])]


In [99]:
decryption = [brmult_list(T,[e,multiplier]).astype(int) for e in encryption]
print(decryption)


[array([ 110,    1,  110,   10,  110,   11,  110,  100,  110,  101,  110,
        110,  110,  111,  110, 1000,  110, 1001,  110, 1010,  110, 1011,
        110, 1100,  110, 1101,  110, 1110,  110, 1111,  111,    0,  111]), array([  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   1, 111,  10, 111,  11])]


In [93]:
def to_bytestr(m, nbytes =2):
    return ''.join(chr(m[i*8:i*8+8],2 for i in range(len(m)//8))

In [94]:
msg_rec = to_bytestr(decryption[0])

TypeError: chr() takes exactly one argument (2 given)